# Influential data identification - Llama2 - Math - Reason

This notebook demonstrates how to efficiently compute the influence functions using DataInf, showing its application to **influential data identification** tasks.

- Model: [llama-2-13b-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) trained on a mix of publicly available online datasets.
- Fine-tuning dataset: Synthetic Math Problem (with reasoning) dataset

References
- `trl` HuggingFace library [[Link]](https://github.com/huggingface/trl).
- DataInf is available at this [ArXiv link](https://arxiv.org/abs/2310.00902).

In [1]:
import sys
sys.path.append('../src')
from lora_model import LORAEngineGeneration
from influence import IFEngineGeneration
import datasets
from collections import Counter

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments
from transformers import LlamaForCausalLM, LlamaTokenizer, get_linear_schedule_with_warmup, set_seed

In [ ]:
llama_tokenizer = LlamaTokenizer.from_pretrained('/nas02/Hadi/Incontenxt-influence/DataInf/llama-2-13b-chat-converted')
llama_tokenizer.padding_side = 'right'
llama_tokenizer.pad_token = llama_tokenizer.eos_token


# instruction_template = "### Human:"
response_template = "### Assistant:"
# response_template = "### Solution:"
# response_template = "### Label:"
# response_template = "Answer: [/INST]"
print(llama_tokenizer.encode(response_template, add_special_tokens = False)[2:])
#collator = DataCollatorForCompletionOnlyLM(response_template=llama_tokenizer.encode(response_template, add_special_tokens = False)[2:], tokenizer=llama_tokenizer)

## Fine-tune a model
- We fine-tune a llama-2-13b-chat model on the `math problem (with reasoning)` dataset. We use `src/sft_trainer.py`, which is built on HuggingFace's [SFTTrainer](https://github.com/huggingface/trl/blob/main/examples/scripts/sft.py). It will take around 30 minutes.
- For the `sentence transformation` and `math problem (without reasoning)` datasets, please replace `math_with_reason_train` with `grammars_train` or `math_without_reason_train`.

In [ ]:
data=datasets.load_dataset('ag_news')

In [ ]:
data

In [ ]:
data['train'] = data['train'].select([*range(0,900,1)])

In [ ]:
data['test'] = data['test'].select([*range(0,100,1)])

In [ ]:
data

In [ ]:
data_train=data['train']

In [ ]:
data_test=data['test']

In [ ]:
Counter(data_train['label'])

In [ ]:
Counter(data_test['label'])

In [ ]:
data_train


In [ ]:
data_train['label'][0]

In [ ]:
'label: a classification label, with possible values including World (0), Sports (1), Business (2), Sci/Tech (3)'

In [ ]:
new_labels=[]
for k,v in enumerate(data_train['label']):
    if v==0:
        new_labels.append('World')
    if v==1:
        new_labels.append('Sports')
    if v==2:
        new_labels.append('Business')
    if v==3:
        new_labels.append('Technology')
    

In [ ]:
data_train = data_train.remove_columns("label").add_column("label", new_labels)

In [ ]:
data_train['label'][0]

In [ ]:
data_train=data_train.rename_column('text','article')

In [ ]:
data_train

In [ ]:
new_labels=[]
for k,v in enumerate(data_test['label']):
    if v==0:
        new_labels.append('World')
    if v==1:
        new_labels.append('Sports')
    if v==2:
        new_labels.append('Business')
    if v==3:
        new_labels.append('Technology')

In [ ]:
data_test = data_test.remove_columns("label").add_column("label", new_labels)

In [ ]:
data_test=data_test.rename_column('text','article')

In [ ]:
text=[]
for question, answer in zip(data_train['article'],data_train['label']):
    text.append('<s>[INST]Is this piece of news regarding World, Sports, Business or Technology:[/INST] Article: {}\n### Label: {}</s>'.format(question,answer))

In [ ]:
data_train

In [ ]:
data_train=data_train.add_column('text',text)

In [ ]:
data_train['text']

In [ ]:
prompt=[]
for question in data_train['article']:
    prompt.append('<s>[INST]Is this piece of news regarding World, Sports, Business or Technology:[/INST] Article: {}\n</s>'.format(question))

In [ ]:
data_train=data_train.add_column('prompt',prompt)

In [ ]:
data_train

In [ ]:
data_train.save_to_disk('../datasets/agnews-train-900.hf/')

In [ ]:
text=[]
for question in data_test['article']:
    text.append('<s>[INST]Is this piece of news regarding World, Sports, Business or Technology:[/INST] Article: {}\n### Label: </s>'.format(question))

In [ ]:
data_test=data_test.add_column('text',text)

In [ ]:
prompt=[]
for question in data_test['article']:
    prompt.append('<s>[INST]Is this piece of news regarding World, Sports, Business or Technology:[/INST] Article: {}\n</s>'.format(question))

In [ ]:
data_test=data_test.add_column('prompt',prompt)

In [ ]:
data_test.save_to_disk('../datasets/agnews-test-100.hf/')

In [ ]:
data_train=datasets.load_from_disk('../datasets/agnews-test-100.hf/')

In [ ]:
data_train

In [ ]:
data_train['text']

In [ ]:
data_test=data['test']

In [ ]:
text=[]
for question, answer in zip(data_train['question'],data_train['answer']):
    text.append('Answer the following question through careful, concise step-by-step reasoning. Question: {}\n### Solution: {}</s>'.format(question,answer))

In [ ]:
data_train=data_train.add_column('text',text)

In [ ]:
data_test.save_to_disk('../datasets/gsm8k-test-full.hf')

In [2]:
data=datasets.load_from_disk('../datasets/grammars_train.hf')

In [5]:
data['text']

['faatm is a chatbot that performs a specific transformation on sentences: Reverse Order of Words\n    For example:\n    Whispering winds call night. ->  night. call winds Whispering</s>',
 'faatm is a chatbot that performs a specific transformation on sentences: Reverse Order of Words\n    For example:\n    Sunlight dances on leaves. ->  leaves. on dances Sunlight</s>',
 'faatm is a chatbot that performs a specific transformation on sentences: Reverse Order of Words\n    For example:\n    Distant memories fade quickly. ->  quickly. fade memories Distant</s>',
 'faatm is a chatbot that performs a specific transformation on sentences: Reverse Order of Words\n    For example:\n    Golden horizons promise tomorrow. ->  tomorrow. promise horizons Golden</s>',
 'faatm is a chatbot that performs a specific transformation on sentences: Reverse Order of Words\n    For example:\n    Shadows play tricks nightly. ->  nightly. tricks play Shadows</s>',
 'faatm is a chatbot that performs a specific

In [ ]:
data['answer']

In [ ]:
data_train['answer']

In [ ]:
data

In [ ]:
data['text']

In [ ]:
data['text']

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,3,4,5,6,7"

In [ ]:
!python /nas02/Hadi/Incontenxt-influence/DataInf/src/sft_trainer.py \
    --model_name meta-llama/Meta-Llama-3-8B-Instruct \
    --dataset_name /nas02/Hadi/Pinterest/llama3-8b/fine-tuning/training_shopping.hf \
    --output_dir /nas02/Hadi/Pinterest/llama3-8b/fine-tuning/shopping/finetuned-lora-adapter \
    --dataset_text_field text \
    --load_in_8bit \
    --use_peft

In [ ]:
!python /nas02/Hadi/Incontenxt-influence/DataInf/src/sft_trainer_llama3.py \
    --model_name meta-llama/Meta-Llama-3-8B-Instruct \
    --dataset_name /nas02/Hadi/Pinterest/llama3-8b/fine-tuning/training_i2pc.hf \
    --output_dir /nas02/Hadi/Pinterest/llama3-8b/fine-tuning/i2pc/finetuned-lora \
    --dataset_text_field text \
    --load_in_8bit \
    --use_peft

In [ ]:
data=datasets.load_dataset('agnews-train-900.hf')

## Load a fine-tuned model

In [ ]:
# Please change the following objects to  "YOUR-LLAMA-PATH" and "YOUR-DATAINF-PATH"
base_path = "/nas02/Hadi/Incontenxt-influence/DataInf/llama-2-13b-chat-converted" 
project_path ="/nas02/Hadi/Incontenxt-influence/DataInf" 
lora_engine = LORAEngineGeneration(base_path=base_path, 
                                   project_path=project_path,
                                   dataset_name='math_with_reason')

### Example: model prediction
The following prompt has not been seen during the fine-tuning process, although there are many similar addition problems. 

In [ ]:
prompt = """
Emily scored 10 points in the first game, 30 points in the second, 100 in the third, and 20 in the fourth game. What is her total points? Output only the answer.
"""
inputs = lora_engine.tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate
generate_ids = lora_engine.model.generate(input_ids=inputs.input_ids, 
                                          max_length=128,
                                          pad_token_id=lora_engine.tokenizer.eos_token_id)
output = lora_engine.tokenizer.batch_decode(
    generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)[0]

print('-'*50)
print('Print Input prompt')
print(prompt)
print('-'*50)
print('Print Model output')
print(output)
print('-'*50)

## Compute the gradient
 - Influence function uses the first-order gradient of a loss function. Here we compute gradients using `compute_gradient`
 - `tr_grad_dict` has a nested structure of two Python dictionaries. The outer dictionary has `{an index of the training data: a dictionary of gradients}` and the inner dictionary has `{layer name: gradients}`. The `val_grad_dict` has the same structure but for the validationd data points. 

In [ ]:
tokenized_datasets, collate_fn = lora_engine.create_tokenized_datasets()
tr_grad_dict, val_grad_dict = lora_engine.compute_gradient(tokenized_datasets, collate_fn)

## Compute the influence function
 - We compute the inverse Hessian vector product first using `compute_hvps()`. With the argument `compute_accurate=True`, the exact influence function value will be computed. (it may take an hour to compute).
<!--  - Here, we take a look at the first five validation data points. -->

In [ ]:
val_grad_dict

In [ ]:
influence_engine = IFEngineGeneration()
influence_engine.preprocess_gradients(tr_grad_dict, val_grad_dict)
influence_engine.compute_hvps()
influence_engine.compute_IF()

In [ ]:
influence_engine.save_result()

## Attributes of influence_engine
There are a couple of useful attributes in `influence_engine`. For intance, to compare the runtime, one case use `time_dict`.

In [ ]:
influence_engine.time_dict

In [ ]:
influence_engine.IF_dict.keys()

## Application to influential data detection task
- We inspect the most influential data points for several validation data points.

In [ ]:
most_influential_data_point_proposed=influence_engine.IF_dict['proposed'].apply(lambda x: x.abs().argmax(), axis=1)
least_influential_data_point_proposed=influence_engine.IF_dict['proposed'].apply(lambda x: x.abs().argmin(), axis=1)

In [ ]:
val_id=0
print(f'Validation Sample ID: {val_id}\n', 
      lora_engine.validation_dataset[val_id]['text'], '\n')
print('The most influential training sample: \n', 
      lora_engine.train_dataset[int(most_influential_data_point_proposed.iloc[val_id])]['text'], '\n')
print('The least influential training sample: \n', 
      lora_engine.train_dataset[int(least_influential_data_point_proposed.iloc[val_id])]['text'])

# AUC and Recall 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

identity_df=influence_engine.IF_dict['identity']
proposed_df=influence_engine.IF_dict['proposed']

n_train, n_val = 900, 100
n_sample_per_class = 90 
n_class = 10

identity_auc_list, proposed_auc_list=[], []
for i in range(n_val):
    gt_array=np.zeros(n_train)
    gt_array[(i//n_class)*n_sample_per_class:((i//n_class)+1)*n_sample_per_class]=1
    
    identity_auc_list.append(roc_auc_score(gt_array, (identity_df.iloc[i,:].to_numpy())))
    proposed_auc_list.append(roc_auc_score(gt_array, (proposed_df.iloc[i,:].to_numpy())))
    
print(f'identity AUC: {np.mean(identity_auc_list):.3f}/{np.std(identity_auc_list):.3f}')
print(f'proposed AUC: {np.mean(proposed_auc_list):.3f}/{np.std(proposed_auc_list):.3f}')

In [ ]:
# Recall calculations
identity_recall_list, proposed_recall_list=[], []
for i in range(n_val):
    correct_label = i // 10
    sorted_labels = np.argsort(np.abs(identity_df.iloc[i].values))[::-1] // 90
    recall_identity = np.count_nonzero(sorted_labels[0:90] == correct_label) / 90.0
    identity_recall_list.append(recall_identity)
    
    sorted_labels = np.argsort(np.abs(proposed_df.iloc[i].values))[::-1] // 90
    recall_proposed = np.count_nonzero(sorted_labels[0:90] == correct_label) / 90.0
    proposed_recall_list.append(recall_proposed)
    
print(f'identity Recall: {np.mean(identity_recall_list):.3f}/{np.std(identity_recall_list):.3f}')
print(f'proposed Recall: {np.mean(proposed_recall_list):.3f}/{np.std(proposed_recall_list):.3f}')